# Sizing the Air-to-Water Cooling Coil of a Multi-Zone VAV System

Let's consider a building having two separately controlled temperature zones. Let's assume that we already did the cooling and heating load calculations of the building. Now, we need to design the VAV system. We do this here only for peak summer design conditions. After the required cooling coil capacity has been determined, we can size the air-to-water cooling coil.

We use the class `VAVSystem` in `hvac.air_conditioning.multi_zone.design` for doing the design calculations of the VAV system. To organize our programming code, we configure the VAV system and run the design calculations in a separate class. Through a method of this class, we then retrieve the cooling coil requirements.

Once we have the cooling coil requirements at our disposal, we choose the entering and the leaving water temperature to determine the required mass flow rate of water through the cooling coil.
 
We know, or we already have selected the heat exchanger core geometry for the cooling coil, but we still need to determine the face area dimensions and the number of rows of the cooling coil. To determine the cooling coil's face width and height, we select the face velocity of the air entering the cooling coil. We also choose a number of rows. Now, the air-to-water cooling coil is completely configured. Let's assume that we can model the air-cooling coil with the class `PlainFinTubeAirToWaterCounterFlowHeatExchanger` in `hvac.heat_exchanger.fintube.continuous_fin` as a plain fin-tube heat exchanger in a counterflow arrangement. Once we have the model of our air-cooling coil, we can set its operating conditions and then retrieve its calculated performance. Finally, we must check if the calculated available cooling capacity of the configured cooling coil matches with the required cooling capacity. If not, we can alter the parameters we selected ourselves, or maybe we will need to select another heat exchanger core geometry. 

First, we import all the things we'll need:

In [1]:
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid
from hvac.air_conditioning.multi_zone import Zone, Season
from hvac.air_conditioning.multi_zone.design import VAVSystem
from hvac.heat_exchanger.recuperator.fintube.continuous_fin import (
    PlainFinTubeAirToWaterCounterFlowHeatExchanger as AirCoil
)

Q_ = Quantity
Water = Fluid('Water')

C:\Users\Tom\PycharmProjects\python-hvac\hvac\fluids\fluid.py:244: CoolPropWarning: CoolProp could not solve for quantity 'sigma: surface tension is only defined within the two-phase region; Try PQ or QT inputs'
  warnings.warn(


## VAV System Design and Cooling Coil Requirements

We define an encapsulating class `AirConditioningSystem` for doing the design calculations of the VAV system inside this class. We configure the building zones as attributes of this class. On instantiation of the class, we create a `VAVSystem` instance and directly call its method `design_summer()`. Through the method `get_cooling_coil_requirements()` of the encapsulating class `AirConditioningSystem` we return a dictionary with all the information that is needed to size the cooling coil.

In [2]:
class AirConditioningSystem:
    zone_A = Zone(
        name='zone A',
        summer=Season(
            Q_sen=Q_(24, 'kW'),
            Q_lat=Q_(6, 'kW'),
            zone_air=HumidAir(Tdb=Q_(26, 'degC'), RH=Q_(50, 'pct'))
        ),
        winter=Season(
            Q_sen=Q_(-11.25, 'kW'),
            Q_lat=Q_(0, 'kW'),
            zone_air=HumidAir(Tdb=Q_(22, 'degC'), RH=Q_(50, 'pct'))
        )
    )
    zone_B = Zone(
        name='zone B',
        summer=Season(
            Q_sen=Q_(15, 'kW'),
            Q_lat=Q_(5, 'kW'),
            zone_air=HumidAir(Tdb=Q_(26, 'degC'), RH=Q_(50, 'pct'))
        ),
        winter=Season(
            Q_sen=Q_(-5, 'kW'),
            Q_lat=Q_(0, 'kW'),
            zone_air=HumidAir(Tdb=Q_(22, 'degC'), RH=Q_(50, 'pct'))
        )
    )

    def __init__(self):
        self.vav_system = VAVSystem(
            zones=[self.zone_A, self.zone_B],
            outdoor_air_summer=HumidAir(Tdb=Q_(28.9, 'degC'), Twb=Q_(20.1, 'degC')),
            outdoor_air_winter=HumidAir(Tdb=Q_(-5.7, 'degC'), Twb=Q_(-6.7, 'degC')),
            V_vent=Q_(1225, 'm**3 / hr'),
        )
        self.vav_system.design_summer()

    def get_cooling_coil_requirements(self):
        return {
            'Q_dot_cc': self.vav_system.summer.cooling_coil.Q,
            'Q_dot_cc_sen': self.vav_system.summer.cooling_coil.Q_sen,
            'air_in': self.vav_system.summer.mixed_air,
            'air_out': self.vav_system.summer.cooled_air,
            'air_m_dot': self.vav_system.summer.m_supply
        }

In [3]:
air_system = AirConditioningSystem()
d = air_system.get_cooling_coil_requirements()
Q_dot_cc = d['Q_dot_cc']
Q_dot_cc_sen = d['Q_dot_cc_sen']
air_in = d['air_in']
air_out = d['air_out']
air_m_dot = d['air_m_dot']
print(
    f"required total cooling capacity: {Q_dot_cc.to('kW'):~P.3f}",
    f"required sensible cooling capacity: {Q_dot_cc_sen.to('kW'):~P.3f}",
    f"entering air = {air_in.Tdb.to('degC'):~P.2f} DB, "
    f"{air_in.W.to('g / kg'):~P.1f} AH, "
    f"{air_in.RH.to('pct'):~P.0f} RH",
    f"leaving air = {air_out.Tdb.to('degC'):~P.2f} DB, "
    f"{air_out.W.to('g / kg'):~P.1f} AH, "
    f"{air_out.RH.to('pct'):~P.0f} RH",
    f"mass flow rate of air = {air_m_dot.to('kg / hr'):~P.3f}",
    sep='\n'
)

required total cooling capacity: -51.884 kW
required sensible cooling capacity: -40.156 kW
entering air = 26.36 °C DB, 10.6 g/kg AH, 49 % RH
leaving air = 14.00 °C DB, 9.2 g/kg AH, 92 % RH
mass flow rate of air = 11470.588 kg/h


## Determine the Water Mass Flow Rate through the Cooling Coil

Select the entering and leaving water temperature to determine the required mass flow rate of water from the required cooling coil capacity.

In [4]:
water_T_in = Q_(8, 'degC')
water_T_out = Q_(13, 'degC')

water_P = Q_(2, 'bar')
water_in = Water(T=water_T_in, P=water_P)
water_out = Water(T=water_T_out, P=water_P)
water_cp_avg = (water_in.cp + water_out.cp) / 2

water_m_dot = -Q_dot_cc / (water_cp_avg * (water_T_out - water_T_in))

print(
    "mass flow rate of water = "
    f"{water_m_dot.to('kg / hr'):~P.3f}"
)

mass flow rate of water = 8906.359 kg/h


## Configure the Air-Cooling Coil Model

We configure the model of the air-cooling coil inside a function. The sizing attributes of the cooling coil are parameters to this function, but the core heat exchanger geometry is embedded inside the function.

In [5]:
def design_air_coil(width: Quantity, height: Quantity, num_rows: int) -> AirCoil:
    air_coil = AirCoil(
        width=width,
        height=height,
        num_rows=num_rows,
        pitch_trv=Q_(25.4, 'mm'),
        pitch_lon=Q_(22.0, 'mm'),
        d_i=Q_(8.422, 'mm'),
        d_o=Q_(10.2, 'mm'),
        t_fin=Q_(0.3302, 'mm'),
        fin_density=1 / Q_(3.175, 'mm'),
        num_circuits=None
    )
    return air_coil

To determine the width and height of the cooling coil's face area, we select a face velocity for the air.

In [6]:
face_velocity = Q_(2, 'm / s')

air_V_dot = air_m_dot / air_in.rho
A_face = air_V_dot / face_velocity
print(
    f"air-cooling coil face area = {A_face.to('m ** 2'):~P.3f}"
)

air_coil_width = Q_(A_face.to('m ** 2').magnitude, 'm')
air_coil_height = Q_(1, 'm')

air-cooling coil face area = 1.374 m²


To complete the configuration, we select the number of rows of the cooling coil, and then we call our function `design_air_coil()`.

In [7]:
num_rows = 5

air_coil = design_air_coil(air_coil_width, air_coil_height, num_rows)

## Determine the Cooling Coil Performance

From the cooling coil requirements, we have the state of air entering the cooling coil and the air mass flow rate at peak summer design conditions. We have selected the state of the water entering the cooling coil, and we have determined the water mass flow rate based on the required cooling capacity of the coil. After setting these operating conditions, we can rate the cooling coil and check the results. Ideally, the available cooling capacity and the resulting state of the air leaving the cooling coil should agree with the requirements. 

In [8]:
air_coil.set_operating_conditions(
    air_in=air_in,
    water_in=water_in,
    air_m_dot=air_m_dot,
    water_m_dot=water_m_dot
)

d = air_coil.rate()

Q_dot_cc = d['Q_dot']
air_out = d['air_out']

print(
    f"cooling coil capacity = {Q_dot_cc.to('kW'):~P.3f}",
    "leaving air = "
    f"{air_out.Tdb.to('degC'):~P.2f} DB, "
    f"{air_out.W.to('g / kg'):~P.2f} AH, "
    f"{air_out.RH.to('pct'):~P.0f} RH",
    sep='\n'
)

cooling coil capacity = 53.756 kW
leaving air = 14.23 °C DB, 8.87 g/kg AH, 87 % RH


By changing parameters (the entering and/or leaving water temperature, the air face velocity, the number of cooling coil rows), we can investigate the effect on the available cooling coil capacity and the state of air leaving the cooling coil.